In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# Load the datasets
All_Journals = pd.read_excel("/Users/adishsundar/Desktop/URA/All_Journals.xlsx")
Finance_Journals = pd.read_excel("/Users/adishsundar/Desktop/URA/Finance_Journals.xlsx")
Management_Journals = pd.read_excel("/Users/adishsundar/Desktop/URA/Management_Journals.xlsx")

# Look at all columns
pd.set_option('display.max_columns', None)

# Look at all rows
pd.set_option('display.max_rows', None)

All_Journals.head()

# Clean Data set and Get Important Columns
new_all_journals = All_Journals[['Source Title', 'Authors', 'Article Title', 'Affiliations', 
                                 'Cited Reference Count', 'Publication Date', 'Publication Year']]

new_finance_journals = Finance_Journals[['Source Title', 'Authors', 'Article Title', 'Affiliations', 
                                         'Cited Reference Count', 'Publication Date', 'Publication Year']]

new_management_journals = Management_Journals[['Source Title', 'Authors', 'Article Title', 'Affiliations', 
                                               'Cited Reference Count', 'Publication Date', 'Publication Year']]

new_all_journals.head()

# 14696 rows in entire DB
len(new_all_journals)

# Create copies of datasets to work with
fin_copy = new_finance_journals.copy()
man_copy = new_management_journals.copy()

# number of rows in finance and management journals
print(len(fin_copy), len(man_copy))

# Comparing cited reference count between Management and Finance Journals

# Total of 350,660 cited references in finance journals
new_finance_journals['Cited Reference Count'].sum()

# Total of 363,541 cited references in management journals
new_management_journals['Cited Reference Count'].sum()

# Looking at number of single-authored, double authored, and three or more authored articles across journals
# Create a new column that stores the # of Authors for every row
fin_copy['#OfAuthors'] = (fin_copy['Authors'].str.count(';')) + 1

man_copy['#OfAuthors'] = (fin_copy['Authors'].str.count(';')) + 1

# Count of 1, 2, and 3 or more authors in finance journals
count = len(fin_copy[fin_copy['#OfAuthors'] == 1])
count2 = len(fin_copy[fin_copy['#OfAuthors'] == 2])
count3 = len(fin_copy[fin_copy['#OfAuthors'] >= 3])

print(count,count2,count3)

# Finanance percentages
print(1894/8723, 3388/8723, 3441/8723)

# Count of 1, 2, and 3 or more authors in management journals
count = len(man_copy[man_copy['#OfAuthors'] == 1])
count2 = len(man_copy[man_copy['#OfAuthors'] == 2])
count3 = len(man_copy[man_copy['#OfAuthors'] >= 3])

print(count,count2,count3)

# Management percentages
print(1105 / 5973, 2289/5973, 2579/5973)

# new row for each finance affiliation, now we can group by affiliations better
fin_copy['Affiliations'] = fin_copy['Affiliations'].str.split(';')
fin_copy = fin_copy.explode('Affiliations')
fin_copy = fin_copy.reset_index(drop=True)

# new row for each management affiliation, now we can group by affiliations better
man_copy['Affiliations'] = man_copy['Affiliations'].str.split(';')
man_copy = man_copy.explode('Affiliations')
man_copy = man_copy.reset_index(drop=True)

print(fin_copy.groupby('Affiliations').size().sort_values(ascending=False))

print(man_copy.groupby('Affiliations').size().sort_values(ascending=False))

# There are NaN Publication values, drop them 
fin_copy_dropped = fin_copy.dropna()
man_copy_dropped = man_copy.dropna()

fin_copy_dropped.head()

df_grouped = fin_copy_dropped.groupby(['Affiliations', 'Publication Year']).size().reset_index(name='count')

df_pivot = df_grouped.pivot(index='Publication Year', columns='Affiliations', values='count').fillna(0)

# Look at number of articles published per year between finance and management journals
fin_copy = new_finance_journals.copy()
man_copy = new_management_journals.copy()

# Articles published per year for Finance Journals
print(fin_copy.groupby('Publication Year').size().sort_values(ascending=False))

# Articles published per year for Management Journals
print(man_copy.groupby('Publication Year').size().sort_values(ascending=False))

# Run a simple regression (probit regression ( 0 - 1 variable as dependent probability scale) independent 
# is reputation of instutuion based off of number of articles published).
# start with 1995 as the 0 year and have a 5 year lag. 
# Comparison between top 6 and other B-tier journals and university presence within them.
# Finance: Journal of Financial and Quantitative Analysis, 
# Management: 

# Running a simple regression with the number of articles an instution has in any given year, and relating that to
# the lag of 1995 being the 0 year.
# also lag of institutions vs lag of authors.

# Left side, pear year number of articles that an institution has in each of the journals 

new_copy = fin_copy_dropped.copy()
new_copy['Affiliations'] = new_copy['Affiliations'].str.split(';')
new_copy = new_copy.explode('Affiliations')
new_copy = new_copy.reset_index(drop=True)
new_copy.head()

x = new_copy.copy()

# Group the data by year and count the number of records in each group
x.rename(columns={"Cited Reference Count": "Cited_Reference_Count", "Publication Year": "Publication_Year"}, inplace=True)
# Number of Articles Per Year
article_counts = x.groupby("Publication_Year").size().reset_index(name="num_Articles_Year")

# Merge the counts column back to the original data frame
z = pd.merge(x, article_counts, on = "Publication_Year")

# Print the updated data frame
z

# create a new column with the number of articles 5 years earlier
z_copy = z.copy()

z_copy['articles_lag'] = z_copy['num_Articles_Year'].shift(5)

# subset the data to exclude the first 5 years
z_copy = z_copy[z_copy['Publication_Year'] >= 2000]

# create a linear regression plot
sns.lmplot(x='articles_lag', y='num_Articles_Year', data=z_copy)

# show the plot
plt.show()

# Create a new column with the number of articles 5 years prior
z_copy = z.copy()

z_copy['articles_lag5'] = z_copy.groupby('Affiliations')['num_Articles_Year'].shift(5)

# Drop any rows with missing data
z_copy.dropna(inplace=True)

# Run the regression
X = sm.add_constant(z_copy['articles_lag5'])
y = z_copy['num_Articles_Year']
model = sm.OLS(y, X)
results = model.fit()

# Print the regression results
print(results.summary())

z_copy = z.copy()

affiliation_count = z_copy.groupby('Affiliations').size().reset_index(name='Affiliation_Reputation')

# Merge the affiliation count data frame back to the original data frame
z = pd.merge(z_copy, affiliation_count, on='Affiliations')
# cols_to_remove = [col for col in z.columns if col.endswith(('_x', '_y'))]
# z = z.drop(cols_to_remove, axis=1)
z.head()

z_copy = z.copy()

# Create the regression model with Affiliation_Reputation, Cited_Reference_Count, and Publication_Year
regression_model = smf.ols(formula='Cited_Reference_Count ~ Publication_Year', data = z_copy)

# Fit the model and print the summary
results = regression_model.fit()
print(results.summary())

z_copy = z.copy()

# Create the regression model with Affiliation_Reputation, Cited_Reference_Count, and Publication_Year
regression_model = smf.ols(formula='Cited_Reference_Count ~ Affiliation_Reputation', data = z_copy)

# Fit the model and print the summary
results = regression_model.fit()
print(results.summary())

z_copy = z.copy()

# Create a linear regression model
model = sm.OLS.from_formula('Cited_Reference_Count ~ Affiliation_Reputation + Publication_Year', data = z_copy)
results = model.fit()

# Print the regression summary
print(results.summary())

# Create a scatter plot with separate lines for each year
sns.lmplot(x='Affiliation_Reputation', y='Cited_Reference_Count', hue='Publication_Year', data = z_copy)

z_copy = z.copy()

# Group data by affiliation and publication year, and compute the mean cited_reference_count for each group
grouped = z_copy.groupby(['Affiliations', 'Publication_Year'])['Cited_Reference_Count'].mean().reset_index()

# Pivot the data to create a matrix where the rows represent affiliations, the columns represent publication years,
# and the values represent the mean cited_reference_count
pivot = grouped.pivot(index='Affiliations', columns='Publication_Year', values='Cited_Reference_Count')

# Only keep affiliations that had a mean cited reference count of over 100 at some point in time
pivot = pivot.loc[(pivot > 100).any(axis=1)]

# Create a line plot showing the change in cited_reference_count over time for each affiliation
pivot.T.plot(figsize=(10, 6))

# Set plot title and axis labels
plt.title('Change in Cited Reference Count by Affiliation over Time')
plt.xlabel('Publication Year')
plt.ylabel('Mean Cited Reference Count')
plt.show()

fin_copy_dropped.head()

